# **BirdCLEF 2025 Training Notebook**

This is a baseline training pipeline for BirdCLEF 2025 using EfficientNetB0 with PyTorch and Timm(for pretrained EffNet). You can check inference and preprocessing notebooks in the following links: 

- [EfficientNet B0 Pytorch [Inference] | BirdCLEF'25](https://www.kaggle.com/code/kadircandrisolu/efficientnet-b0-pytorch-inference-birdclef-25)

  
- [Transforming Audio-to-Mel Spec. | BirdCLEF'25](https://www.kaggle.com/code/kadircandrisolu/transforming-audio-to-mel-spec-birdclef-25)  

Note that by default this notebook is in Debug Mode, so it will only train the model with 2 epochs, but the [weight](https://www.kaggle.com/datasets/kadircandrisolu/birdclef25-effnetb0-starter-weight) I used in the inference notebook was obtained after 10 epochs of training.

**Features**
* Implement with Pytorch and Timm
* Flexible audio processing with both pre-computed and on-the-fly mel spectrograms
* Stratified 5-fold cross-validation with ensemble capability
* Mixup training for improved generalization
* Spectrogram augmentations (time/frequency masking, brightness adjustment)
* AdamW optimizer with Cosine Annealing LR scheduling
* Debug mode for quick experimentation with smaller datasets

**Pre-computed Spectrograms**
For faster training, you can use pre-computed mel spectrograms from [this dataset](https://www.kaggle.com/datasets/kadircandrisolu/birdclef25-mel-spectrograms) by setting `LOAD_DATA = True`

## Libraries

In [1]:
# === ライブラリのインポート ===
# 基本的なライブラリ
import os  # OS操作（ファイルパスなど）
import logging  # ログ出力用
import random  # 乱数生成
import gc  # ガベージコレクション（メモリ管理）
import time  # 時間計測
import cv2  # OpenCV（画像処理、ここではリサイズに使用）
import math  # 数学演算
import warnings  # 警告メッセージの制御
from pathlib import Path  # オブジェクト指向のファイルパス操作

# データ操作・数値計算
import numpy as np  # 数値計算（配列操作）
import pandas as pd  # データフレーム操作

# 機械学習・評価
from sklearn.model_selection import StratifiedKFold  # 層化K分割交差検証
from sklearn.metrics import roc_auc_score  # ROC AUCスコア計算

# 音声処理
import librosa  # 音声分析ライブラリ (メルスペクトログラム計算など)

# 深層学習 (PyTorch)
import torch  # PyTorch本体
import torch.nn as nn  # ニューラルネットワークのモジュール（層、損失関数など）
import torch.nn.functional as F  # よく使う関数（活性化関数など）
import torch.optim as optim  # 最適化アルゴリズム (Adam, SGDなど)
from torch.optim import lr_scheduler  # 学習率スケジューラ
from torch.utils.data import Dataset, DataLoader  # データセット、データローダー

# 可視化
import matplotlib.pyplot as plt  # グラフ描画
import seaborn as sns  # 統計的グラフ描画

# ユーティリティ
from tqdm.auto import tqdm  # プログレスバー表示
import timm  # PyTorch Image Models (事前学習済みモデルライブラリ) <= ★これ便利！

# 警告を非表示に設定
warnings.filterwarnings("ignore")
# ログレベルをERROR以上に設定（INFOやWARNINGを表示しない）
logging.basicConfig(level=logging.ERROR)

# augument用
import albumentations as A
from albumentations.pytorch import ToTensorV2 # Tensor変換も任せる場合
from albumentations.core.transforms_interface import ImageOnlyTransform



## Configuration

In [2]:
# In [2]: CFG クラス (サブセット対応版)
class CFG:
    # --- 基本設定 ---
    seed = 42
    # debug = True # <<< debug フラグはエポック数/Fold数制御に使う (データサイズは別途指定)
    print_freq = 100
    num_workers = 2 # <<< エラーが出る場合は 0 にする

    # --- パス設定 ---
    OUTPUT_DIR = '/kaggle/working/'
    train_datadir = '/kaggle/input/birdclef-2025/train_audio'
    train_csv = '/kaggle/input/birdclef-2025/train.csv'
    taxonomy_csv = '/kaggle/input/birdclef-2025/taxonomy.csv'

    # ★★★ 使用する NPY ファイルのパスを指定 ★★★
    # 例: spectrogram_npy = '/kaggle/input/your-subset-npy-dataset/your_subset_file.npy'
    # 例: spectrogram_npy = '/kaggle/input/your-full-npy-dataset/your_full_file.npy'
    spectrogram_npy = '/kaggle/input/melspec-m128-h512-f50-14000-s256x256-subset5000/melspec_rand5s_M192_H320_F50-15000_S256x256_subset5000.npy' # <<< ★必ず使用するNPYファイルのパスに書き換える！

    # --- データ処理設定 ---
    LOAD_DATA = True  # <<< NPYファイルを使うので True 固定
    # ↓↓↓ .npyファイル生成時の設定に合わせておく（ドキュメント目的） ↓↓↓
    FS = 32000
    TARGET_DURATION = 5.0
    TARGET_SHAPE = (256, 256) # <<< ★使用するNPYの形状に合わせる！
    N_FFT = 1024
    HOP_LENGTH = 320
    N_MELS = 192
    FMIN = 50
    FMAX = 14000
    # ↑↑↑ .npyファイル生成時の設定に合わせておく（ドキュメント目的） ↑↑↑

    # --- ★★★ データセットサイズ設定 ★★★ ---
    USE_SUBSET = True    # <<< サブセットNPYを使う場合は True, フルNPYなら False
    SUBSET_SIZE = 5000    # <<< USE_SUBSET=True の時の目安サイズ（実際はNPY依存）
                          # この値自体はフィルタリング後のサイズ決定には使わない

    # --- モデル設定 ---
    model_name = 'efficientnet_b0'
    pretrained = True
    in_channels = 1

    # --- 学習設定 ---
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    epochs = 15           # <<< 学習エポック数（デバッグ時は下で上書き可能）
    batch_size = 32
    criterion = 'BCEWithLogitsLoss'

    # --- ★★★ Focal Loss パラメータを追加 ★★★ ---
    focal_loss_alpha = 0.25 # デフォルト値 (調整可能)
    focal_loss_gamma = 2.0  # デフォルト値 (調整可能)
    # ------------------------------------

    # --- 交差検証 設定 ---
    n_fold = 5
    selected_folds = [0, 1, 2, 3, 4] # <<< 実行したいFoldを指定（デバッグ時は下で上書き可能）

    # --- 最適化 設定 ---
    optimizer = 'AdamW'
    lr = 5e-4
    weight_decay = 1e-5

    # --- スケジューラ 設定 ---
    scheduler = 'CosineAnnealingLR'
    min_lr = 1e-6
    T_max = epochs

    # --- データ拡張・Mixup 設定 ---
    aug_prob = 0.5
    mixup_alpha = 0.5

        # --- データ拡張・Mixup 設定 ---
    # aug_prob = 0.5 # <<< 古いAugmentation用。Albumentations内のpを使うなら不要かも？
    mixup_alpha = 0.5

    # === ★★★ カスタムSpecAugment用パラメータを追加 ★★★ ===
    custom_spec_augment_p = 0.5  # カスタムSpecAugmentを適用する確率
    custom_spec_augment_num_mask = 2 # マスクの基本数 (1からこの値までの間でランダムに選ばれる)
    custom_spec_augment_freq_max_pct = 0.15 # 周波数マスクの最大割合
    custom_spec_augment_time_max_pct = 0.20 # 時間マスクの最大割合
    # ===========================================

    # === ★★★ Albumentations用パラメータを追加 ★★★ ===
    # Coarse Dropout (Random Erasing)
    coarse_dropout_p = 0  # 適用確率
    coarse_dropout_max_holes = 8
    coarse_dropout_max_height_pct = 0.15 # 高さの割合
    coarse_dropout_max_width_pct = 0.15  # 幅の割合

    # Brightness/Contrast
    brightness_contrast_p = 0.5
    brightness_limit = 0.1
    contrast_limit = 0.1

    # Gaussian Noise
    gauss_noise_p = 0
    gauss_var_limit = (10.0, 50.0)
    # ===========================================

    # --- デバッグ用設定 ---
    # 別のフラグでデバッグモードを管理 (例: 少ないエポック数、Fold 0 のみ実行)
    DEBUG_TRAIN = True # <<< True にすると少ないEpoch/Foldで実行

    def update_debug_settings(self):
        # DEBUG_TRAIN フラグに基づいてEpochとFold数を制限
        if self.DEBUG_TRAIN:
            print("!!! Running in DEBUG_TRAIN mode: epochs=15, selected_folds=[0] !!!")
            self.epochs = 15  # 例: デバッグ時のエポック数
            self.selected_folds = [0] # 例: デバッグ時に実行するFold

# インスタンス作成
cfg = CFG()
# デバッグ設定を適用
cfg.update_debug_settings()

!!! Running in DEBUG_TRAIN mode: epochs=15, selected_folds=[0] !!!


## Utilities

In [3]:
# === ユーティリティ関数 ===

def set_seed(seed=42):
    """
    再現性のために乱数シードを固定する関数
    """
    # Python標準のrandomモジュール
    random.seed(seed)
    # ハッシュのシード (辞書のキー順序などに関わる)
    os.environ["PYTHONHASHSEED"] = str(seed)
    # NumPyの乱数シード
    np.random.seed(seed)
    # PyTorchのCPU乱数シード
    torch.manual_seed(seed)
    # PyTorchの現在のGPU乱数シード
    torch.cuda.manual_seed(seed)
    # PyTorchの全GPU乱数シード (複数のGPUがある場合)
    torch.cuda.manual_seed_all(seed)
    # cuDNNの決定論的アルゴリズムを使用する設定 (Trueにすると再現性が上がるが遅くなる可能性)
    torch.backends.cudnn.deterministic = True
    # cuDNNのベンチマークモードを無効化 (Trueだと最適なアルゴリズムを探すが結果が変わる可能性)
    torch.backends.cudnn.benchmark = False

# シードを固定
set_seed(cfg.seed)

## Pre-processing
These functions handle the transformation of audio files to mel spectrograms for model input, with flexibility controlled by the `LOAD_DATA` parameter. The process involves either loading pre-computed spectrograms from this [dataset](https://www.kaggle.com/datasets/kadircandrisolu/birdclef25-mel-spectrograms) (when `LOAD_DATA=True`) or dynamically generating them (when `LOAD_DATA=False`), transforming audio data into spectrogram representations, and preparing it for the neural network.

In [4]:
# === 前処理関数 ===
# 音声データからメルスペクトログラムを生成する関数群

def audio2melspec(audio_data, cfg):
    """
    音声データ (NumPy配列) をメルスペクトログラムに変換する関数
    """
    # もしデータにNaNが含まれていたら、平均値で補完 (念のため)
    if np.isnan(audio_data).any():
        mean_signal = np.nanmean(audio_data)
        audio_data = np.nan_to_num(audio_data, nan=mean_signal)

    # librosaを使ってメルスペクトログラムを計算
    mel_spec = librosa.feature.melspectrogram(
        y=audio_data,         # 入力音声データ
        sr=cfg.FS,            # サンプリング周波数
        n_fft=cfg.N_FFT,      # FFT窓サイズ
        hop_length=cfg.HOP_LENGTH, # ホップ長
        n_mels=cfg.N_MELS,    # メルフィルタバンク数
        fmin=cfg.FMIN,        # 最小周波数
        fmax=cfg.FMAX,        # 最大周波数
        power=2.0             # パワースペクトログラム (**2) を計算
    )

    # パワースペクトログラムをデシベル(dB)スケールに変換
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    # デシベルスケールのスペクトログラムを0-1の範囲に正規化
    mel_spec_norm = (mel_spec_db - mel_spec_db.min()) / (mel_spec_db.max() - mel_spec_db.min() + 1e-8) # ゼロ除算防止

    return mel_spec_norm

def process_audio_file(audio_path, cfg):
    """
    単一の音声ファイルを処理してメルスペクトログラムを取得する関数
    (LOAD_DATA=False の場合、Datasetクラスから呼ばれる)
    """
    try:
        # librosaで音声ファイルをロード (指定サンプリング周波数でリサンプリングされる)
        audio_data, _ = librosa.load(audio_path, sr=cfg.FS)

        # 目標サンプル数を計算 (目標時間 * サンプリング周波数)
        target_samples = int(cfg.TARGET_DURATION * cfg.FS)

        # 音声が目標サンプル数より短い場合、繰り返して長さを合わせる
        if len(audio_data) < target_samples:
            n_copy = math.ceil(target_samples / len(audio_data)) # 必要な繰り返し回数
            if n_copy > 1:
                audio_data = np.concatenate([audio_data] * n_copy) # 配列を指定回数連結

        # 音声の中央から目標サンプル数分を抽出 (5秒クリップ)
        start_idx = max(0, int(len(audio_data) / 2 - target_samples / 2)) # 開始インデックス
        end_idx = min(len(audio_data), start_idx + target_samples) # 終了インデックス
        center_audio = audio_data[start_idx:end_idx] # 中央部分をスライス

        # 抽出後、長さが足りない場合 (ほぼ起こらないはずだが念のため)、ゼロパディング
        if len(center_audio) < target_samples:
            center_audio = np.pad(center_audio,
                                 (0, target_samples - len(center_audio)),
                                 mode='constant') # 末尾を0で埋める

        # 中央5秒の音声データをメルスペクトログラムに変換
        mel_spec = audio2melspec(center_audio, cfg)

        # スペクトログラムの形状が目標形状と異なる場合、リサイズ
        # (librosaの計算結果は厳密には目標形状にならないことがあるため)
        if mel_spec.shape != cfg.TARGET_SHAPE:
            mel_spec = cv2.resize(mel_spec, cfg.TARGET_SHAPE, interpolation=cv2.INTER_LINEAR) # 線形補間でリサイズ

        # float32形式で返す (PyTorchのTensorにしやすい)
        return mel_spec.astype(np.float32)

    except Exception as e:
        # エラーが発生した場合、ログを出力してNoneを返す
        print(f"Error processing {audio_path}: {e}")
        return None

def generate_spectrograms(df, cfg):
    """
    データフレーム内の全音声ファイルからメルスペクトログラムを生成する関数
    (このノートブックでは LOAD_DATA=True のため、実際にはあまり使われない)
    """
    print("Generating mel spectrograms from audio files...")
    start_time = time.time() # 開始時間記録

    all_bird_data = {} # 結果を格納する辞書 {samplename: spectrogram}
    errors = [] # エラーが発生したファイル情報を格納するリスト

    # データフレームを行ごとに処理 (tqdmでプログレスバー表示)
    for i, row in tqdm(df.iterrows(), total=len(df)):
        # デバッグモードの場合、1000件処理したら終了
        if cfg.debug and i >= 1000:
            break

        try:
            samplename = row['samplename'] # サンプル名取得
            filepath = row['filepath'] # ファイルパス取得

            # 音声ファイルを処理してスペクトログラムを取得
            mel_spec = process_audio_file(filepath, cfg)

            # 処理が成功したら辞書に格納
            if mel_spec is not None:
                all_bird_data[samplename] = mel_spec

        except Exception as e:
            # 予期せぬエラーをキャッチ
            print(f"Error processing {row.filepath}: {e}")
            errors.append((row.filepath, str(e)))

    end_time = time.time() # 終了時間記録
    # 処理結果のサマリを出力
    print(f"Processing completed in {end_time - start_time:.2f} seconds")
    print(f"Successfully processed {len(all_bird_data)} files out of {len(df)}")
    print(f"Failed to process {len(errors)} files")

    return all_bird_data # 生成したスペクトログラムの辞書を返す

## Dataset Preparation and Data Augmentations
We'll convert audio to mel spectrograms and apply random augmentations with 50% probability each - including time stretching, pitch shifting, and volume adjustments. This randomized approach creates diverse training samples from the same audio files

In [5]:
# # In [5]: Dataset クラス (デバッグ用サンプリングを削除)
# class BirdCLEFDatasetFromNPY(Dataset):
#     def __init__(self, df, cfg, spectrograms, mode="train"): # <<< spectrograms を直接受け取るように変更
#         """
#         コンストラクタ
#         Args:
#             df (pd.DataFrame): このFoldで使用するデータフレーム (サブセット/フル)
#             cfg (CFG): 設定オブジェクト
#             spectrograms (dict): ロード済みのスペクトログラム辞書
#             mode (str, optional): "train" または "valid". Defaults to "train".
#         """
#         self.df = df # このFold/モード用のデータフレーム
#         self.cfg = cfg
#         self.mode = mode
#         self.spectrograms = spectrograms # ロード済みの辞書を保持

#         # --- 分類情報関連 (変更なし) ---
#         taxonomy_df = pd.read_csv(self.cfg.taxonomy_csv)
#         self.species_ids = taxonomy_df['primary_label'].tolist()
#         self.num_classes = len(self.species_ids)
#         self.label_to_idx = {label: idx for idx, label in enumerate(self.species_ids)}

#         # --- ★★★ デバッグ用の self.df のサンプリング処理を削除 ★★★ ---
#         # if cfg.debug:
#         #     self.df = self.df.sample(min(1000, len(self.df)), random_state=cfg.seed).reset_index(drop=True)
#         # --- (削除ここまで) ---

#         # --- samplename が df にあるか確認 (念のため) ---
#         if 'samplename' not in self.df.columns:
#              # もし samplename がなければここでエラーにするか、生成ロジックを入れる
#              # (run_training で生成しているので通常は不要なはず)
#              raise ValueError("DataFrame passed to Dataset must contain 'samplename' column.")

#         # --- マッチング件数表示 (オプション) ---
#         # sample_names_in_df = set(self.df['samplename'])
#         # found_samples = sum(1 for name in sample_names_in_df if name in self.spectrograms)
#         # print(f"Dataset '{mode}': Using {len(self.df)} samples. Found {found_samples} matching spectrograms in provided dict.")


#     def __len__(self):
#         return len(self.df)

#     def __getitem__(self, idx):
#         row = self.df.iloc[idx]
#         samplename = row['samplename'] # samplename を取得
#         spec = None

#         # --- NPYからロードする処理 ---
#         # self.spectrograms (ロード済みの辞書) に samplename が存在するかチェック
#         if samplename in self.spectrograms:
#             spec = self.spectrograms[samplename]
#         # (LOAD_DATA=True 前提なので elif not self.cfg.LOAD_DATA: は不要)

#         # --- スペクトログラムが見つからなかった場合の処理 ---
#         # (NPYに含まれるキーに基づいてDFをフィルタリングしたので、基本ここには来ないはずだが念のため)
#         if spec is None:
#             print(f"CRITICAL WARNING: Spectrogram for {samplename} not found in provided dictionary for mode '{self.mode}'. Returning zeros.")
#             spec = np.zeros(self.cfg.TARGET_SHAPE, dtype=np.float32)
#             # ここでエラーを発生させる方が安全かもしれない
#             # raise KeyError(f"Spectrogram for {samplename} not found in provided dictionary!")

#         # --- 以降は変更なし ---
#         spec = torch.tensor(spec, dtype=torch.float32).unsqueeze(0)
#         if self.mode == "train" and random.random() < self.cfg.aug_prob:
#             spec = self.apply_spec_augmentations(spec)
#         target = self.encode_label(row['primary_label'])
#         if 'secondary_labels' in row and row['secondary_labels'] not in [[''], None, np.nan]:
#             if isinstance(row['secondary_labels'], str):
#                 try: secondary_labels = eval(row['secondary_labels'])
#                 except: secondary_labels = []
#             else: secondary_labels = row['secondary_labels']
#             for label in secondary_labels:
#                 if label in self.label_to_idx: target[self.label_to_idx[label]] = 1.0
#         return {
#             'melspec': spec,
#             'target': torch.tensor(target, dtype=torch.float32),
#             'filename': row['filename']
#         }

#     # --- apply_spec_augmentations, encode_label は変更なし ---
# # === データ拡張関数 (変更なし) ===
#     def apply_spec_augmentations(self, spec):
#         """ スペクトログラムにデータ拡張を適用する関数 """
#         # 時間マスキング
#         if random.random() < 0.5:
#             num_masks = random.randint(1, 3)
#             for _ in range(num_masks):
#                 # width = random.randint(5, 20) # 元のコードの幅
#                 t = int(spec.shape[2] * 0.05) # 例: 時間軸の5%程度の幅
#                 width = random.randint(t // 2, t) if t > 0 else 1
#                 start = random.randint(0, spec.shape[2] - width)
#                 spec[0, :, start:start+width] = 0
#         # 周波数マスキング
#         if random.random() < 0.5:
#             num_masks = random.randint(1, 3)
#             for _ in range(num_masks):
#                 # height = random.randint(5, 20) # 元のコードの高さ
#                 f = int(spec.shape[1] * 0.1) # 例: 周波数軸の10%程度の高さ
#                 height = random.randint(f // 2, f) if f > 0 else 1
#                 start = random.randint(0, spec.shape[1] - height)
#                 spec[0, start:start+height, :] = 0
#         # 明るさ/コントラスト調整 (元のコードではコメントアウトされていた？必要なら有効化)
#         # if random.random() < 0.5:
#         #     gain = random.uniform(0.8, 1.2)
#         #     bias = random.uniform(-0.1, 0.1)
#         #     spec = spec * gain + bias
#         #     spec = torch.clamp(spec, 0, 1)
#         return spec

#     # === ラベルエンコード関数 (これが重要！) ===
#     def encode_label(self, label):
#         """ ラベル文字列を Multi-hot ベクトルにエンコードする関数 """
#         # クラス数分のゼロベクトルを作成 
#         target = np.zeros(self.num_classes, dtype=np.float32) # dtypeも指定推奨
#         # ラベルが既知のクラスなら、対応するインデックスを1.0にする
#         if label in self.label_to_idx:
#             target[self.label_to_idx[label]] = 1.0
#         # 作成した target ベクトルを返す
#         return target

In [6]:
# CustomSpecAugment クラス定義 (セル全体をこれで置き換え)
import albumentations as A
from albumentations.core.transforms_interface import ImageOnlyTransform
import random
import numpy as np

class CustomSpecAugment(ImageOnlyTransform):
    def __init__(self, 
                 num_mask=2, 
                 freq_masking_max_percentage=0.15, 
                 time_masking_max_percentage=0.20, 
                 fill_value=0, 
                 always_apply=False, # ★デフォルトは False (ブール値)
                 p=0.5):             # ★デフォルトは 0.5 (float値)
        
        # 親クラスのコンストラクタを正しい引数の順序で呼び出す
        super().__init__(always_apply=always_apply, p=p) # ★キーワード引数で明示的に指定
        
        self.num_mask = num_mask
        self.freq_masking_max_percentage = freq_masking_max_percentage
        self.time_masking_max_percentage = time_masking_max_percentage
        self.fill_value = fill_value

    def apply(self, img, **params): # img は NumPy 配列 (H, W) を想定
        img_aug = img.copy()
        num_actual_masks = random.randint(1, self.num_mask)

        for _ in range(num_actual_masks):
            all_freqs_num, all_frames_num = img_aug.shape

            # 周波数マスキング
            freq_percentage_to_mask = random.uniform(0.0, self.freq_masking_max_percentage)
            num_freqs_to_mask = int(freq_percentage_to_mask * all_freqs_num)
            if num_freqs_to_mask > 0:
                f0 = np.random.uniform(low=0.0, high=max(1, all_freqs_num - num_freqs_to_mask)) # highがlowより小さくならないように
                f0 = int(f0)
                img_aug[f0:min(all_freqs_num, f0 + num_freqs_to_mask), :] = self.fill_value # 境界チェック

            # 時間マスキング
            time_percentage_to_mask = random.uniform(0.0, self.time_masking_max_percentage)
            num_frames_to_mask = int(time_percentage_to_mask * all_frames_num)
            if num_frames_to_mask > 0:
                t0 = np.random.uniform(low=0.0, high=max(1, all_frames_num - num_frames_to_mask)) # highがlowより小さくならないように
                t0 = int(t0)
                img_aug[:, t0:min(all_frames_num, t0 + num_frames_to_mask)] = self.fill_value # 境界チェック
        
        return img_aug

    def get_transform_init_args_names(self):
        return ("num_mask", "freq_masking_max_percentage", "time_masking_max_percentage", "fill_value")

print("CustomSpecAugment class defined (fully replaced version).")

CustomSpecAugment class defined (fully replaced version).


In [7]:
# BirdCLEFDatasetFromNPY クラス定義 (セル全体をこれで置き換え)
# (必要な import 文: Dataset, pd, np, torch, A, CustomSpecAugment は既に実行されている想定)

class BirdCLEFDatasetFromNPY(Dataset):
    def __init__(self, df, cfg, spectrograms, mode="train"):
        self.df = df
        self.cfg = cfg
        self.mode = mode
        self.spectrograms = spectrograms

        taxonomy_df = pd.read_csv(self.cfg.taxonomy_csv)
        self.species_ids = taxonomy_df['primary_label'].tolist()
        self.num_classes = len(self.species_ids)
        self.label_to_idx = {label: idx for idx, label in enumerate(self.species_ids)}

        if self.mode == 'train':
            self.augmentations = A.Compose([
                CustomSpecAugment( # ★キーワード引数で全て明示的に指定
                    num_mask=cfg.custom_spec_augment_num_mask,
                    freq_masking_max_percentage=cfg.custom_spec_augment_freq_max_pct,
                    time_masking_max_percentage=cfg.custom_spec_augment_time_max_pct,
                    fill_value=0,
                    always_apply=False, # 明示的に False を指定 (デフォルトを使っても良い)
                    p=cfg.custom_spec_augment_p
                ),
                A.CoarseDropout(
                    max_holes=cfg.coarse_dropout_max_holes,
                    max_height=int(cfg.TARGET_SHAPE[0] * cfg.coarse_dropout_max_height_pct),
                    max_width=int(cfg.TARGET_SHAPE[1] * cfg.coarse_dropout_max_width_pct),
                    fill_value=0,
                    p=cfg.coarse_dropout_p
                ),
                A.RandomBrightnessContrast(
                    brightness_limit=cfg.brightness_limit,
                    contrast_limit=cfg.contrast_limit,
                    p=cfg.brightness_contrast_p
                ),
                A.GaussNoise(
                    var_limit=cfg.gauss_var_limit,
                    p=cfg.gauss_noise_p
                ),
            ])
        else:
            self.augmentations = None

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        samplename = row['samplename']
        spec = None

        if self.spectrograms and samplename in self.spectrograms:
            spec = self.spectrograms[samplename].astype(np.float32)
        else:
            print(f"CRITICAL WARNING: Spectrogram for {samplename} not found in NPY dict for mode '{self.mode}'. Returning zeros.")
            spec = np.zeros(self.cfg.TARGET_SHAPE, dtype=np.float32)

        if self.augmentations:
            augmented = self.augmentations(image=spec)
            spec = augmented['image']

        spec = torch.tensor(spec, dtype=torch.float32).unsqueeze(0)

        target = self.encode_label(row['primary_label'])
        if 'secondary_labels' in row and row['secondary_labels'] not in [[''], None, np.nan]:
            secondary_labels_val = row['secondary_labels']
            if isinstance(secondary_labels_val, str):
                try: 
                    processed_secondary_labels = eval(secondary_labels_val)
                except: 
                    processed_secondary_labels = []
            else: 
                processed_secondary_labels = secondary_labels_val
            
            for label in processed_secondary_labels: # 変数名を変更
                if label in self.label_to_idx:
                    target[self.label_to_idx[label]] = 1.0

        return {
            'melspec': spec,
            'target': torch.tensor(target, dtype=torch.float32),
            'filename': row['filename']
        }

    def encode_label(self, label):
        target = np.zeros(self.num_classes, dtype=np.float32)
        if label in self.label_to_idx:
            target[self.label_to_idx[label]] = 1.0
        return target

print("BirdCLEFDatasetFromNPY class defined (fully replaced version).")

BirdCLEFDatasetFromNPY class defined (fully replaced version).


In [8]:
# === カスタム Collate 関数 ===
# DataLoaderがバッチを作成する際に、サンプルリストをどのようにまとめるかを定義する関数

def collate_fn(batch):
    """
    バッチ内のサンプルを適切にまとめるカスタムcollate関数
    (Datasetの__getitem__がNoneを返す可能性を考慮しているが、現状はゼロ埋めのため、
     主にテンソルのスタックが目的)
    """
    # バッチからNoneの要素を除去 (エラーでNoneが返された場合などに対応)
    batch = [item for item in batch if item is not None]
    # もしバッチが空になったら空の辞書を返す
    if len(batch) == 0:
        return {}

    # バッチ内の最初のサンプルのキーを使って結果用の辞書を初期化
    result = {key: [] for key in batch[0].keys()}

    # 各サンプルの値を対応するキーのリストに追加
    for item in batch:
        for key, value in item.items():
            result[key].append(value)

    # キーごとに値を適切に処理
    for key in result:
        # 'target' キーの値がTensorなら、リスト内のTensorをスタックしてバッチ次元を作る
        if key == 'target' and isinstance(result[key][0], torch.Tensor):
            result[key] = torch.stack(result[key])
        # 'melspec' キーの値がTensorなら、同様にスタック
        elif key == 'melspec' and isinstance(result[key][0], torch.Tensor):
            # (念のため) バッチ内の全スペクトログラムの形状が同じか確認してからスタック
            shapes = [t.shape for t in result[key]]
            if len(set(str(s) for s in shapes)) == 1: # 全形状が同じなら
                result[key] = torch.stack(result[key])
            # もし形状が異なっていたら... (現状のリサイズ処理では起こりにくいはず)
            # ここではリストのまま返すか、エラーを出すか等の処理が必要になるが、
            # このコードでは形状が同じ前提でスタックのみ行っている

    return result # まとめたバッチ(辞書)を返す

## Model Definition

In [9]:
# === モデル定義 ===
# BirdCLEFタスク用のPyTorchモデルクラス

class BirdCLEFModel(nn.Module):
    def __init__(self, cfg):
        """コンストラクタ"""
        super().__init__() # 親クラス(nn.Module)のコンストラクタ呼び出し
        self.cfg = cfg # 設定オブジェクトを保持

        # 分類情報CSVからクラス数を取得
        taxonomy_df = pd.read_csv(cfg.taxonomy_csv)
        cfg.num_classes = len(taxonomy_df) # クラス数をcfgに追加（または上書き）

        # timmライブラリを使って事前学習済みモデルをロード
        self.backbone = timm.create_model(
            cfg.model_name,           # モデル名 (例: 'efficientnet_b0')
            pretrained=cfg.pretrained,# 事前学習済み重みを使うか
            in_chans=cfg.in_channels, # 入力チャネル数 (1 = グレースケール)
            drop_rate=0.2,            # ドロップアウト率 (全結合層の手前)
            drop_path_rate=0.2        # DropPath率 (Stochastic Depth)
        )

        # モデルの種類に応じて、最終層(分類層)を削除し、その入力特徴量を保持
        if 'efficientnet' in cfg.model_name:
            # EfficientNetの場合、classifier層の入力特徴量数を取得
            backbone_out = self.backbone.classifier.in_features
            # 元のclassifier層をIdentity（何もしない層）に置き換え
            self.backbone.classifier = nn.Identity()
        elif 'resnet' in cfg.model_name:
            # ResNet系の場合、fc層の入力特徴量数を取得
            backbone_out = self.backbone.fc.in_features
            # 元のfc層をIdentityに置き換え
            self.backbone.fc = nn.Identity()
        else: # その他のtimmモデルの場合 (汎用的な取得方法)
            backbone_out = self.backbone.get_classifier().in_features
            # 元の分類層をリセット (クラス数を0に)
            self.backbone.reset_classifier(0, '')

        # Global Average Pooling層 (特徴マップをベクトルに変換)
        self.pooling = nn.AdaptiveAvgPool2d(1)

        # 特徴ベクトルの次元数
        self.feat_dim = backbone_out

        # 新しい分類層 (バックボーンからの特徴量を入力とし、クラス数を出力)
        self.classifier = nn.Linear(backbone_out, cfg.num_classes)

        # Mixupが設定で有効になっているか確認
        self.mixup_enabled = hasattr(cfg, 'mixup_alpha') and cfg.mixup_alpha > 0
        if self.mixup_enabled:
            # Mixupのアルファ値を保持
            self.mixup_alpha = cfg.mixup_alpha

    def forward(self, x, targets=None):
        """
        順伝播メソッド
        Args:
            x (torch.Tensor): 入力テンソル (バッチサイズ, 1, 高さ, 幅)
            targets (torch.Tensor, optional): ターゲットテンソル (Mixup用). Defaults to None.
        Returns:
            torch.Tensor: モデルの出力ロジット (バッチサイズ, クラス数)
            or tuple(torch.Tensor, torch.Tensor): Mixup有効な訓練時は (ロジット, Mixup損失)
        """

        # --- Mixup処理 (学習時かつMixup有効の場合) ---
        if self.training and self.mixup_enabled and targets is not None:
            # mixup_dataメソッドで入力とターゲットをミックス
            mixed_x, targets_a, targets_b, lam = self.mixup_data(x, targets)
            x = mixed_x # 入力をミックスしたものに置き換え
        else:
            # Mixupしない場合、関連変数をNoneに
            targets_a, targets_b, lam = None, None, None

        # --- 特徴抽出 ---
        # バックボーンモデルに入力を通して特徴量を取得
        features = self.backbone(x)

        # 一部のtimmモデルは辞書形式で特徴量を返すことがあるため対応
        if isinstance(features, dict):
            features = features['features'] # 想定されるキーから取得

        # 特徴量が4次元テンソル (B, C, H, W) の場合、プーリングしてベクトル化
        if len(features.shape) == 4:
            features = self.pooling(features) # (B, C, 1, 1)
            features = features.view(features.size(0), -1) # (B, C) に変形

        # --- 分類 ---
        # 最終的な分類層（線形層）に入力し、ロジット（活性化関数適用前の出力）を得る
        logits = self.classifier(features)

        # --- Mixup損失計算 (学習時かつMixup有効の場合) ---
        if self.training and self.mixup_enabled and targets is not None:
            # mixup_criterionメソッドでミックスされたターゲットに対する損失を計算
            loss = self.mixup_criterion(F.binary_cross_entropy_with_logits,
                                       logits, targets_a, targets_b, lam)
            # ロジットと計算済み損失のタプルを返す
            return logits, loss

        # 通常時 (Mixupなし、または推論時) はロジットのみを返す
        return logits

    def mixup_data(self, x, targets):
        """ Mixupを実行するメソッド """
        batch_size = x.size(0) # バッチサイズ取得

        # ベータ分布からミックス比率ラムダ(lam)をサンプリング
        lam = np.random.beta(self.mixup_alpha, self.mixup_alpha)

        # バッチ内でシャッフルするためのインデックスを作成
        indices = torch.randperm(batch_size).to(x.device)

        # 入力データをミックス (x = lam * x_a + (1 - lam) * x_b)
        mixed_x = lam * x + (1 - lam) * x[indices]
        # 対応するターゲットも取得 (ミックスはしない、それぞれのターゲットを保持)
        targets_a, targets_b = targets, targets[indices]

        return mixed_x, targets_a, targets_b, lam

    def mixup_criterion(self, criterion, pred, y_a, y_b, lam):
        """ Mixup時の損失関数を計算するメソッド """
        # 損失 = lam * Loss(pred, y_a) + (1 - lam) * Loss(pred, y_b)
        return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

## Training Utilities
We are configuring our optimization strategy with the AdamW optimizer, cosine scheduling, and the BCEWithLogitsLoss criterion.

In [10]:
class BCEFocalLoss(nn.Module):
    """
    マルチラベル用の Focal Loss (BCEWithLogitsLoss ベースで数値的に安定)
    参考: https://www.kaggle.com/code/thedrcat/focal-multilabel-loss-in-pytorch-explained
         (ただし、最後のスケーリングは標準的な mean reduction に修正)
    """
    def __init__(self, alpha=0.25, gamma=2.0, reduction='mean'):
        super().__init__()
        # alpha: クラス間の不均衡を調整する係数 (0~1)。Positiveクラスの重み。
        self.alpha = alpha
        # gamma: 簡単なサンプルの損失への寄与を抑制する度合い (>=0)。大きいほど難しいサンプル重視。
        self.gamma = gamma
        # reduction: 出力損失の集約方法 ('mean', 'sum', 'none')
        self.reduction = reduction

    def forward(self, logits, targets):
        # logits: モデルの出力 (Sigmoid適用前) - shape: (N, C)
        # targets: 正解ラベル (Multi-hotベクトル) - shape: (N, C)

        # BCEWithLogitsLoss を使って、要素ごとの基本的なロス (-log(pt)) を安定して計算
        bce_loss = F.binary_cross_entropy_with_logits(logits, targets, reduction='none')

        # 予測確率 pt を計算 (pt = p if target=1, 1-p if target=0)
        probas = torch.sigmoid(logits) # まずSigmoidで確率pを計算
        # targets が 1 の箇所は probas を、0 の箇所は 1-probas を使う
        pt = targets * probas + (1 - targets) * (1 - probas)

        # Focal Loss の変調係数 (1 - pt)**gamma を計算
        # pt の値が小さい（＝予測が難しい・間違っている）ほど、この係数は1に近くなる
        # pt の値が大きい（＝予測が簡単・合っている）ほど、この係数は0に近づく
        modulating_factor = torch.pow(1.0 - pt, self.gamma)

        # alphaによる重み付け係数を計算 (alpha_t = alpha if target=1, 1-alpha if target=0)
        alpha_weight_factor = targets * self.alpha + (1 - targets) * (1 - self.alpha)

        # 最終的な Focal Loss を要素ごとに計算
        # focal_loss = alpha_t * (1 - pt)**gamma * bce_loss
        focal_loss = alpha_weight_factor * modulating_factor * bce_loss

        # 指定された方法でロスを集約
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:  # 'none'
            return focal_loss

print("BCEFocalLoss class defined.")

BCEFocalLoss class defined.


In [11]:
# === 学習用ユーティリティ関数 ===
# オプティマイザ、スケジューラ、損失関数を生成するファクトリ関数

def get_optimizer(model, cfg):
    """ 設定に基づいてオプティマイザを生成する """
    if cfg.optimizer == 'Adam':
        optimizer = optim.Adam(
            model.parameters(), # 最適化対象のモデルパラメータ
            lr=cfg.lr,          # 学習率
            weight_decay=cfg.weight_decay # 重み減衰
        )
    elif cfg.optimizer == 'AdamW': # <= 今回はこれ
        optimizer = optim.AdamW(
            model.parameters(),
            lr=cfg.lr,
            weight_decay=cfg.weight_decay
        )
    elif cfg.optimizer == 'SGD':
        optimizer = optim.SGD(
            model.parameters(),
            lr=cfg.lr,
            momentum=0.9, # モーメンタム
            weight_decay=cfg.weight_decay
        )
    else: # 未対応のオプティマイザ名の場合
        raise NotImplementedError(f"Optimizer {cfg.optimizer} not implemented")

    return optimizer

def get_scheduler(optimizer, cfg):
    """ 設定に基づいて学習率スケジューラを生成する """
    if cfg.scheduler == 'CosineAnnealingLR': # <= 今回はこれ
        scheduler = lr_scheduler.CosineAnnealingLR(
            optimizer,      # 対象のオプティマイザ
            T_max=cfg.T_max,  # 1サイクルのエポック数
            eta_min=cfg.min_lr # 最小学習率
        )
    elif cfg.scheduler == 'ReduceLROnPlateau': # 検証ロスが改善しなくなったらLRを下げる
        scheduler = lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode='min',     # 'min' (ロス最小化) or 'max' (精度最大化)
            factor=0.5,     # 学習率を減らす係数 (new_lr = lr * factor)
            patience=2,     # 何エポック改善しなかったら発動するか
            min_lr=cfg.min_lr,# 最小学習率
            verbose=True    # 発動時にメッセージ表示
        )
    elif cfg.scheduler == 'StepLR': # 指定エポックごとにLRを下げる
        scheduler = lr_scheduler.StepLR(
            optimizer,
            step_size=cfg.epochs // 3, # 何エポックごとにLRを下げるか
            gamma=0.5          # 学習率を減らす係数 (new_lr = lr * gamma)
        )
    elif cfg.scheduler == 'OneCycleLR': # OneCycleLR (別の場所で設定される想定)
        scheduler = None # ここではNoneを返す
    else: # スケジューラなし、または未対応の場合
        scheduler = None

    return scheduler

def get_criterion(cfg):
    """ 設定に基づいて損失関数を生成する """
    if cfg.criterion == 'BCEWithLogitsLoss':
        criterion = nn.BCEWithLogitsLoss()
        print("Using BCEWithLogitsLoss")
    elif cfg.criterion == 'FocalLoss':
        # === ★★★ ここで BCEFocalLoss クラスをインスタンス化 ★★★ ===
        criterion = BCEFocalLoss(alpha=cfg.focal_loss_alpha, gamma=cfg.focal_loss_gamma, reduction='mean')
        print(f"Using BCEFocalLoss with alpha={cfg.focal_loss_alpha}, gamma={cfg.focal_loss_gamma}")
        # ====================================================
    else: # 未対応の損失関数名の場合
        raise NotImplementedError(f"Criterion {cfg.criterion} not implemented")

    return criterion

## Training Loop

In [12]:
# === 学習・検証ループ関数 ===

def train_one_epoch(model, loader, optimizer, criterion, device, scheduler=None):
    """ 1エポック分の学習処理を行う関数 """

    model.train() # モデルを学習モードに設定 (Dropoutなどが有効になる)
    losses = [] # 各バッチの損失を格納するリスト
    all_targets = [] # 全ターゲットラベルを格納するリスト
    all_outputs = [] # 全モデル出力を格納するリスト

    # データローダーからバッチを取得してループ (tqdmでプログレスバー表示)
    pbar = tqdm(enumerate(loader), total=len(loader), desc="Training")

    for step, batch in pbar:

        # --- バッチ処理 (CollateFnがリストを返す可能性への対応 - 若干冗長かも) ---
        # もしバッチの'melspec'がリスト形式だったら (通常はTensorのはず)
        if isinstance(batch['melspec'], list):
            # (この部分は現状のCollateFnではあまり通らない想定だが、念のため)
            batch_outputs = []
            batch_losses = []

            # リスト内の各サンプルを個別に処理
            for i in range(len(batch['melspec'])):
                # 1サンプル分の入力とターゲットを取得し、デバイスに転送
                inputs = batch['melspec'][i].unsqueeze(0).to(device)
                target = batch['target'][i].unsqueeze(0).to(device)

                optimizer.zero_grad() # 勾配を初期化
                output = model(inputs) # 順伝播
                loss = criterion(output, target) # 損失計算
                loss.backward() # 誤差逆伝播

                batch_outputs.append(output.detach().cpu()) # 出力をCPUに戻して保存
                batch_losses.append(loss.item()) # 損失値を保存

            optimizer.step() # パラメータ更新 (全サンプル処理後)
            outputs = torch.cat(batch_outputs, dim=0).numpy() # 出力を結合
            loss = np.mean(batch_losses) # 損失の平均値
            targets = batch['target'].numpy() # ターゲット (すでにCPUにある想定)

        # --- 通常のバッチ処理 (入力がTensorの場合) ---
        else:
            # 入力とターゲットをデバイスに転送
            inputs = batch['melspec'].to(device)
            targets = batch['target'].to(device)

            optimizer.zero_grad() # 勾配を初期化
            # 順伝播 (Mixup有効時は (logits, loss) のタプルが返る)
            outputs = model(inputs, targets if model.training and model.mixup_enabled else None)

            # Mixup有効時 (タプルが返ってくる)
            if isinstance(outputs, tuple):
                outputs, loss = outputs # ロジットと損失をアンパック
            # Mixup無効時 (ロジットのみ返ってくる)
            else:
                loss = criterion(outputs, targets) # 損失を計算

            loss.backward() # 誤差逆伝播
            optimizer.step() # パラメータ更新

            # 結果をCPUに戻してNumPy配列に変換
            outputs = outputs.detach().cpu().numpy()
            targets = targets.detach().cpu().numpy()

        # OneCycleLRスケジューラの場合、ステップごとに更新
        if scheduler is not None and isinstance(scheduler, lr_scheduler.OneCycleLR):
            scheduler.step()

        # --- 結果の保存 ---
        all_outputs.append(outputs) # モデル出力をリストに追加
        all_targets.append(targets) # ターゲットラベルをリストに追加
        losses.append(loss if isinstance(loss, float) else loss.item()) # 損失値をリストに追加

        # プログレスバーに現在の損失と学習率を表示
        pbar.set_postfix({
            'train_loss': np.mean(losses[-10:]) if losses else 0, # 直近10バッチの平均損失
            'lr': optimizer.param_groups[0]['lr'] # 現在の学習率
        })

    # --- エポック終了後の処理 ---
    # 全バッチの出力とターゲットを結合
    all_outputs = np.concatenate(all_outputs)
    all_targets = np.concatenate(all_targets)
    # AUCスコアを計算
    auc = calculate_auc(all_targets, all_outputs)
    # 平均損失を計算
    avg_loss = np.mean(losses)

    # 平均損失とAUCスコアを返す
    return avg_loss, auc


def validate(model, loader, criterion, device):
    """ 1エポック分の検証処理を行う関数 """

    model.eval() # モデルを評価モードに設定 (Dropoutなどが無効になる)
    losses = [] # 損失リスト
    all_targets = [] # ターゲットリスト
    all_outputs = [] # 出力リスト

    # 勾配計算を無効化 (メモリ節約＆計算高速化)
    with torch.no_grad():
        # 検証データローダーでループ (tqdmでプログレスバー表示)
        for batch in tqdm(loader, desc="Validation"):
            # --- バッチ処理 (train_one_epochと同様のリスト対応) ---
            if isinstance(batch['melspec'], list):
                batch_outputs = []
                batch_losses = []
                for i in range(len(batch['melspec'])):
                    inputs = batch['melspec'][i].unsqueeze(0).to(device)
                    target = batch['target'][i].unsqueeze(0).to(device)
                    output = model(inputs) # 順伝播 (評価モードなのでMixupはされない)
                    loss = criterion(output, target) # 損失計算
                    batch_outputs.append(output.detach().cpu())
                    batch_losses.append(loss.item())
                outputs = torch.cat(batch_outputs, dim=0).numpy()
                loss = np.mean(batch_losses)
                targets = batch['target'].numpy()
            # --- 通常のバッチ処理 ---
            else:
                inputs = batch['melspec'].to(device)
                targets = batch['target'].to(device)
                outputs = model(inputs) # 順伝播
                loss = criterion(outputs, targets) # 損失計算
                outputs = outputs.detach().cpu().numpy()
                targets = targets.detach().cpu().numpy()

            # 結果をリストに追加
            all_outputs.append(outputs)
            all_targets.append(targets)
            losses.append(loss if isinstance(loss, float) else loss.item())

    # --- 検証終了後の処理 ---
    # 全バッチの結果を結合
    all_outputs = np.concatenate(all_outputs)
    all_targets = np.concatenate(all_targets)

    # AUCスコアを計算
    auc = calculate_auc(all_targets, all_outputs)
    # 平均損失を計算
    avg_loss = np.mean(losses)

    # 平均損失とAUCスコアを返す
    return avg_loss, auc

def calculate_auc(targets, outputs):
    """ ターゲットとモデル出力から平均クラス別AUCを計算する関数 """
    num_classes = targets.shape[1] # クラス数を取得
    aucs = [] # 各クラスのAUCを格納するリスト

    # モデル出力(ロジット)をシグモイド関数で確率に変換
    probs = 1 / (1 + np.exp(-outputs))

    # 各クラスごとにAUCを計算
    for i in range(num_classes):
        # そのクラスの陽性サンプルが存在する場合のみ計算 (存在しないとAUC計算不可)
        if np.sum(targets[:, i]) > 0:
            try:
                # sklearnのroc_auc_scoreで計算
                class_auc = roc_auc_score(targets[:, i], probs[:, i])
                aucs.append(class_auc)
            except ValueError: # AUCが計算できないケース (例:全サンプルが同じクラス)
                pass # スキップ

    # 計算できたクラスのAUCの平均値を返す (計算できなかった場合は0.0)
    return np.mean(aucs) if aucs else 0.0

## Training!

In [13]:
# In [10]: run_training 関数 (データ準備部分を修正)

def run_training(cfg): # <<< 引数から df を削除 (中で full_train_df を読むため)
    """
    交差検証を使ってモデル学習全体を実行する関数
    指定されたNPYファイルをロードして使用する前提
    """
    # --- 基本設定・情報取得 ---
    taxonomy_df = pd.read_csv(cfg.taxonomy_csv)
    species_ids = taxonomy_df['primary_label'].tolist()
    cfg.num_classes = len(species_ids)

    # デバッグ設定適用 (Epoch/Fold数制限)
    # cfg.update_debug_settings() # <<< これは main ブロックで呼ぶ方が良いかも

    # --- ★★★ データ準備 (ここを大きく変更) ★★★ ---
    spectrograms = None
    if cfg.LOAD_DATA:
        print("Loading pre-computed mel spectrograms from NPY file...")
        try:
            # 指定されたNPYファイルをロード
            spectrograms = np.load(cfg.spectrogram_npy, allow_pickle=True).item()
            print(f"Loaded {len(spectrograms)} pre-computed mel spectrograms from {cfg.spectrogram_npy}")
            if len(spectrograms) == 0:
                 raise ValueError("Loaded spectrogram dictionary is empty!")
        except Exception as e:
            print(f"Error loading spectrogram NPY file: {e}")
            print("Cannot continue without spectrogram data. Please check cfg.spectrogram_npy path.")
            return # データがないと学習できないので終了
    else:
        # LOAD_DATA=False はこの修正版では基本的に非推奨
        print("Warning: LOAD_DATA is False. On-the-fly generation logic might be outdated.")
        # (もしオンデマンドを使いたい場合は、Dataset内のロジックが最新か要確認)
        # return # またはエラーにする

    # --- 全訓練メタデータの読み込み ---
    print("Loading full training metadata...")
    full_train_df = pd.read_csv(cfg.train_csv)
    # samplename 列を追加 (Datasetで必要)
    if 'filename' not in full_train_df.columns:
         raise ValueError("train.csv must contain 'filename' column.")
    if 'samplename' not in full_train_df.columns:
        print("Generating 'samplename' column...")
        full_train_df['samplename'] = full_train_df['filename'].map(lambda x: x.split('/')[0] + '-' + x.split('/')[-1].split('.')[0])

    # --- NPYファイルに含まれるキーに基づいてメタデータを選択 ---
    available_samples = set(spectrograms.keys())
    print(f"Filtering metadata based on {len(available_samples)} keys found in the NPY file...")
    # NPYファイルに存在するsamplenameを持つ行だけを抽出
    df_to_use = full_train_df[full_train_df['samplename'].isin(available_samples)].reset_index(drop=True)
    print(f"Using {len(df_to_use)} samples found in both metadata and NPY file.")

    if len(df_to_use) == 0:
        print("Error: No matching samples found between metadata and NPY file. Check paths and file contents.")
        return

    # --- ★★★ K-Fold 分割 (フィルタリング後のデータフレームを使用) ★★★ ---
    print(f"Setting up {cfg.n_fold}-Fold StratifiedKFold...")
    skf = StratifiedKFold(n_splits=cfg.n_fold, shuffle=True, random_state=cfg.seed)
    # df_to_use に対して split を実行
    fold_splits = list(skf.split(df_to_use, df_to_use['primary_label']))

    # --- (以降、Foldループ内のデータ分割部分も修正が必要) ---

    best_scores = [] # CVスコア記録用

    # --- Foldごとのループ ---
    for fold, (train_idx, val_idx) in enumerate(fold_splits):
        if fold not in cfg.selected_folds:
            continue

        print(f'\n{"="*30} Fold {fold} {"="*30}')

        # ★ フィルタリング後の df_to_use から訓練/検証データを抽出 ★
        train_fold_df = df_to_use.iloc[train_idx].reset_index(drop=True)
        val_fold_df = df_to_use.iloc[val_idx].reset_index(drop=True)

        print(f'Training set for Fold {fold}: {len(train_fold_df)} samples')
        print(f'Validation set for Fold {fold}: {len(val_fold_df)} samples')

        # --- DatasetとDataLoaderの作成 (spectrograms 辞書を渡す) ---
        # ★ df は train_fold_df / val_fold_df を使う ★
        train_dataset = BirdCLEFDatasetFromNPY(train_fold_df, cfg, spectrograms=spectrograms, mode='train')
        val_dataset = BirdCLEFDatasetFromNPY(val_fold_df, cfg, spectrograms=spectrograms, mode='valid')

        train_loader = DataLoader(
            train_dataset, batch_size=cfg.batch_size, shuffle=True,
            num_workers=cfg.num_workers, pin_memory=True, collate_fn=collate_fn, drop_last=True
        )
        val_loader = DataLoader(
            val_dataset, batch_size=cfg.batch_size, shuffle=False,
            num_workers=cfg.num_workers, pin_memory=True, collate_fn=collate_fn
        )

        # --- (モデル初期化、学習ループなどは基本的に変更なし) ---
        model = BirdCLEFModel(cfg).to(cfg.device)
        optimizer = get_optimizer(model, cfg)
        criterion = get_criterion(cfg)
        # ... (スケジューラ初期化) ...
        if cfg.scheduler == 'OneCycleLR': # <<< OneCycleLR の設定例も入れておく
             scheduler = lr_scheduler.OneCycleLR(
                 optimizer, max_lr=cfg.lr, steps_per_epoch=len(train_loader), epochs=cfg.epochs, pct_start=0.1
             )
        else:
             scheduler = get_scheduler(optimizer, cfg)


        best_auc = 0
        best_epoch = 0
        # --- エポックループ ---
        for epoch in range(cfg.epochs):
            # ... (train_one_epoch, validate 呼び出し) ...
            train_loss, train_auc = train_one_epoch(model, train_loader, optimizer, criterion, cfg.device, scheduler if isinstance(scheduler, lr_scheduler.OneCycleLR) else None)
            val_loss, val_auc = validate(model, val_loader, criterion, cfg.device)

            # ... (スケジューラ更新、結果表示、モデル保存) ...
            if scheduler is not None and not isinstance(scheduler, lr_scheduler.OneCycleLR):
                 if isinstance(scheduler, lr_scheduler.ReduceLROnPlateau): scheduler.step(val_loss)
                 else: scheduler.step()
            print(f"Epoch {epoch+1}/{cfg.epochs} - Train Loss: {train_loss:.4f}, Train AUC: {train_auc:.4f} | Val Loss: {val_loss:.4f}, Val AUC: {val_auc:.4f}")
            # --- ベストスコア更新 & モデル保存 ---
            # この if 文のブロック全体を正しくインデントする
            if val_auc > best_auc: # 検証AUCが過去最高なら
                # ↓↓↓ この行から下のブロックは if 文の中なので1段階インデント (例: スペース4つ)
                best_auc = val_auc # ベストスコア更新
                best_epoch = epoch + 1 # ベストエポック更新
                print(f"  ---> New best AUC: {best_auc:.4f} at epoch {best_epoch} (Saving model...)")

                # torch.save も if 文の中なので同じインデントレベル
                torch.save({
                    # ↓↓↓ 辞書の中身はさらに1段階インデント (例: 合計スペース8つ)
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'scheduler_state_dict': scheduler.state_dict() if scheduler else None,
                    'epoch': epoch,
                    'val_auc': val_auc,
                    'train_auc': train_auc,
                    'cfg': cfg
                # 辞書の閉じ括弧 '}' は、キーと同じインデントレベル
                }, f"model_fold{fold}.pth") # torch.save の閉じ括弧 ')' は torch.save と同じインデントレベル
            # --- if ブロックの終わり ---
        # --- (Fold終了処理、メモリ解放) ---
        best_scores.append(best_auc)
        print(f"\nBest AUC for fold {fold}: {best_auc:.4f} at epoch {best_epoch}")
        del model, optimizer, scheduler, train_loader, val_loader, train_dataset, val_dataset
        torch.cuda.empty_cache()
        gc.collect()

    # --- (全Fold終了後の結果表示 - 変更なし) ---
    print("\n" + "="*60)
    print("Cross-Validation Results:")
    # ... (省略) ...
    print(f"Mean AUC across {len(best_scores)} folds: {np.mean(best_scores):.4f}")
    print("="*60)


# セル In [11] (メイン実行ブロック)
if __name__ == "__main__":
    import time
    print("\nStarting training process...")
    # --- CFGインスタンス作成とデバッグ設定適用をここで行う ---
    cfg = CFG()
    cfg.update_debug_settings() # デバッグ設定を適用
    # ----------------------------------------------------

    print(f"LOAD_DATA is set to {cfg.LOAD_DATA}")
    if cfg.LOAD_DATA:
        print(f"Attempting to load NPY from: {cfg.spectrogram_npy}")
    else:
         print("LOAD_DATA is False. Expecting on-the-fly generation.")

    # メインの学習関数を実行 (df は渡さない)
    run_training(cfg)

    print("\nTraining complete!")


Starting training process...
!!! Running in DEBUG_TRAIN mode: epochs=15, selected_folds=[0] !!!
LOAD_DATA is set to True
Attempting to load NPY from: /kaggle/input/melspec-m128-h512-f50-14000-s256x256-subset5000/melspec_rand5s_M192_H320_F50-15000_S256x256_subset5000.npy
Loading pre-computed mel spectrograms from NPY file...
Loaded 5000 pre-computed mel spectrograms from /kaggle/input/melspec-m128-h512-f50-14000-s256x256-subset5000/melspec_rand5s_M192_H320_F50-15000_S256x256_subset5000.npy
Loading full training metadata...
Generating 'samplename' column...
Filtering metadata based on 5000 keys found in the NPY file...
Using 5000 samples found in both metadata and NPY file.
Setting up 5-Fold StratifiedKFold...

============================== Fold 0 ==============================
Training set for Fold 0: 4000 samples
Validation set for Fold 0: 1000 samples


model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

Using BCEWithLogitsLoss


Training:   0%|          | 0/125 [00:00<?, ?it/s]

Validation:   0%|          | 0/32 [00:00<?, ?it/s]

...

Validation:   0%|          | 0/32 [00:00<?, ?it/s]

Epoch 14/15 - Train Loss: 0.0209, Train AUC: 0.7627 | Val Loss: 0.0208, Val AUC: 0.8791
  ---> New best AUC: 0.8791 at epoch 14 (Saving model...)


Training:   0%|          | 0/125 [00:00<?, ?it/s]

Validation:   0%|          | 0/32 [00:00<?, ?it/s]

Epoch 15/15 - Train Loss: 0.0211, Train AUC: 0.7456 | Val Loss: 0.0208, Val AUC: 0.8785

Best AUC for fold 0: 0.8791 at epoch 14

Cross-Validation Results:
Mean AUC across 1 folds: 0.8791

Training complete!
